# Regressione non Lineare (parte 2)

**Programmazione di Applicazioni Data Intensive**  
Laurea in Ingegneria e Scienze Informatiche  
DISI - Università di Bologna, Cesena

Proff. Gianluca Moro, Roberto Pasolini  
`nome.cognome@unibo.it`

## Setup

- Importare i package necessari

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

- In Jupyter, configurare l'output di matplotlib integrato nel notebook

In [2]:
%matplotlib inline

## Caso di studio: Predizione dei prezzi delle case

- Riprendiamo dalla scorsa esercitazione il dataset relativo ai prezzi delle case
- Forniamo tale dataset all'URL https://git.io/fjGjx già adattato per essere caricato con `read_csv` con le opzioni di default

In [3]:
import os.path
if not os.path.exists("housing.csv"):
    from urllib.request import urlretrieve
    urlretrieve("https://git.io/fjGjx", "housing.csv")

In [4]:
housing = pd.read_csv("housing.csv")

### Lista delle variabili

- CRIM: tasso di criminalità pro capite per zona
- ZN: proporzione terreno residenziale per lotti maggiori di 25.000 piedi quadrati (circa 2300 m2)
- INDUS: proporzione di acri industriali non commerciali per città
- CHAS: variabile fittizia Charles River, 1 se il tratto affianca il fiume, altrimenti 0
- NOX: concentrazione di ossido d’azoto (parti per 10 milioni)
- RM: numero medio di stanze per abitazione
- AGE: proporzione delle unità abitate costruite prima del 1940
- DIS: distanze pesate verso i cinque uffici di collocamento di Boston
- RAD: indice di accessibilità rispetto alle grandi vie radiali di comunicazione
- TAX: tasso di imposte sulla casa per 10.000 dollari
- PTRATIO: rapporto allievi-docenti per città
- B: 1000(Bk - 0.63)2, dove Bk è la proporzione di persone di origine afroamericana
- LSTAT: percentuale di popolazione con basso reddito
- **MEDV: valore mediano delle abitazioni di proprietà in migliaia di dollari**
  - vogliamo stimare il valore di questa variabile in funzione delle altre

In [5]:
housing.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


- Estraiamo dal frame
  - la serie `y` con i valori della variabile `MEDV` da prevedere
  - il frame `X` con i valori di tutte le altre variabili, utilizzabili per la predizione

In [6]:
y = housing["MEDV"]
X = housing.drop(columns="MEDV")

## Divisione tra training e validation set

- Dividiamo i dati caricati casualmente in
  - un _training set_ per addestrare i modelli
  - un _validation set_ disgiunto su cui verificare l'accuratezza del modello
- Usiamo la funzione `train_test_split` di scikit-learn
  - con `test_size` indichiamo quanti dati vanno nel validation set, i restanti andranno nel training set
  - con `random_state` fissiamo un seed per la suddivisione casuale
  - la funzione mescola i dati di `X` e `y` in modo congiunto, mantenendo la corrispondenza esistente tra le posizioni dei dati

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = \
    train_test_split(X, y, test_size=1/3, random_state=42)

## Funzioni per la validazione

- Riprendiamo dalla prima parte le funzioni per estrarre e stampare le misure di valutazione del modello (MSE, errore relativo, R²)

In [8]:
# importo MSE e R²
from sklearn.metrics import mean_squared_error, r2_score

# definisco funzione per errore relativo
def relative_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true))

# funzione per calcolare e stampare tutte e tre
def print_eval(X, y, model):
    preds = model.predict(X)
    print("   Mean squared error: {:.5}".format(mean_squared_error(y, preds)))
    print("       Relative error: {:.5%}".format(relative_error(y, preds)))
    print("R-squared coefficient: {:.5}".format(r2_score(y, preds)))

## Esercizio 1: Ripasso modelli

Addestrare sul training set creato sopra tre modelli diversi con le seguenti configurazioni, per ciascuno stampare le misure di valutazione sul validation set e i coefficienti associati a ciascuna variabile

- **(1a)** regressione lineare semplice
- **(1b)** regressione lineare con regolarizzazione L2 (regressione ridge), assumendo $\alpha=1$
- **(1c)** regressione lineare con standardizzazione delle feature

Infine:

- **(1d)** visualizzare in un unico frame i coefficienti di tutti e tre i modelli (una riga per variabile, una colonna per modello)

In [10]:
# sono eseguiti quì tutti gli import necessari da scikit-learn
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [11]:
model_a = LinearRegression()
model_a.fit(X_train, y_train)
print_eval(X_val, y_val, model_a)

   Mean squared error: 20.595
       Relative error: 16.20789%
R-squared coefficient: 0.72621


In [12]:
pd.Series(model_a.coef_, index=X.columns)

CRIM       -0.129470
ZN          0.037960
INDUS       0.060978
CHAS        3.213498
NOX       -16.499614
RM          3.911519
AGE        -0.012602
DIS        -1.427742
RAD         0.239546
TAX        -0.008180
PTRATIO    -0.935991
B           0.011948
LSTAT      -0.546562
dtype: float64

In [13]:
model_b = Ridge(alpha=10)
model_b.fit(X_train, y_train)
print_eval(X_val, y_val, model_b)

   Mean squared error: 21.615
       Relative error: 16.55757%
R-squared coefficient: 0.71265


In [14]:
pd.Series(model_b.coef_, index=X.columns)

CRIM      -0.120803
ZN         0.041276
INDUS     -0.003992
CHAS       2.143922
NOX       -1.462094
RM         3.632845
AGE       -0.022260
DIS       -1.207228
RAD        0.220856
TAX       -0.010126
PTRATIO   -0.784938
B          0.012374
LSTAT     -0.606337
dtype: float64

In [15]:
model_c = Pipeline([
    ("scale", StandardScaler()),
    ("lr", LinearRegression())
])
model_c.fit(X_train, y_train)
print_eval(X_val, y_val, model_c)

   Mean squared error: 20.595
       Relative error: 16.20789%
R-squared coefficient: 0.72621


In [16]:
pd.Series(model_c.named_steps["lr"].coef_, index=X.columns)

CRIM      -0.995690
ZN         0.872971
INDUS      0.424808
CHAS       0.857462
NOX       -1.943856
RM         2.820988
AGE       -0.351939
DIS       -3.063072
RAD        2.069621
TAX       -1.356887
PTRATIO   -2.098051
B          1.060004
LSTAT     -3.926623
dtype: float64

In [17]:
pd.DataFrame({
    "linear": model_a.coef_,
    "ridge": model_b.coef_,
    "scaled": model_c.named_steps["lr"].coef_
}, index=X.columns)

,linear,ridge,scaled
CRIM,-0.129470,-0.120803,-0.995690
ZN,0.037960,0.041276,0.872971
INDUS,0.060978,-0.003992,0.424808
CHAS,3.213498,2.143922,0.857462
NOX,-16.499614,-1.462094,-1.943856
RM,3.911519,3.632845,2.820988
AGE,-0.012602,-0.022260,-0.351939
DIS,-1.427742,-1.207228,-3.063072
RAD,0.239546,0.220856,2.069621
TAX,-0.008180,-0.010126,-1.356887


- In tutti e tre i modelli, dai segni dei coefficienti possiamo vedere quali fenomeni influiscono positivamente e negativamente sul prezzo
  - ad es. il prezzo delle case è più alto se vicine al fiume (`CHAS`), mentre decresce con la criminalità (`CRIM`)
- Nella regressione ridge i valori assoluti più alti sono ridotti (es. `NOX` e `RM`)
- Con la standardizzazione delle feature otteniamo valori su scale simili, che possiamo confrontare alla pari
  - ad es. negli altri modelli il coefficiente di `NOX` è alto perché i valori di tale variabile sono bassi (la media è circa 0.55, contro quelle superiori a 3 delle altre variabili)
  - nel modello con standardizzazione assumono invece più peso il numero di stanza (`RM`) e la distanza dagli uffici di collocamento (`DIS`)

## Regressione Lasso

- La regolarizzazione L2 impedisce che i parametri del modello assumano valori troppo alti
- I valori dei parametri sono comunque tutti non nulli, tutte le variabili vengono coinvolte nella predizione
- Vorremmo addestrare un modello meno complesso, dove alcuni parametri hanno valori nulli, **ignorando completamente le variabili meno rilevanti**
  - ad es. variabili con valori dipendenti da altre (_multicollinearità_)
- Questo si può ottenere tramite la regolarizzazione L1, basata sulla norma 1, definita su un vettore $\mathbf{x}$ di $n$ elementi come
$$ \left\Vert\mathbf{x}\right\Vert_1 = \sum_{i=1}^n{\left\vert x_i\right\vert} = \left\vert x_1\right\vert+\ldots+\left\vert x_n\right\vert $$

- La regressione _lasso_ consiste nella regressione lineare con regolarizzazione L1, basata quindi sul minimizzare la funzione d'errore
$$ E = \frac{1}{2m}\left\Vert\mathbf{X}\mathbf{\theta}-\mathbf{y}\right\Vert_2^2 + \alpha\left\Vert\mathbf{\theta}\right\Vert_1 $$
- Come per la regressione ridge, il parametro $\alpha$ controlla il peso della regolarizzazione
- La regressione lasso si esegue usando un modello `Lasso`, su cui possiamo impostare come in `Ridge` il parametro `alpha`

In [18]:
from sklearn.linear_model import Lasso
model = Pipeline([
    ("scale", StandardScaler()),
    ("regr", Lasso(alpha=2))
])
model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('regr',
                 Lasso(alpha=2, copy_X=True, fit_intercept=True, max_iter=1000,
                       normalize=False, positive=False, precompute=False,
                       random_state=None, selection='cyclic', tol=0.0001,
                       warm_start=False))],
         verbose=False)

- Vediamo i coefficienti del modello risultante

In [19]:
pd.Series(model.named_steps["regr"].coef_, X.columns)

CRIM      -0.000000
ZN         0.000000
INDUS     -0.000000
CHAS       0.000000
NOX       -0.000000
RM         2.137434
AGE       -0.000000
DIS        0.000000
RAD       -0.000000
TAX       -0.000000
PTRATIO   -0.571281
B          0.000000
LSTAT     -3.490547
dtype: float64

- La regolarizzazione L1 ha contribuito ad annullare quanti più coefficienti possibile, creando un modello che considera solo 3 variabili
- Ma qual'è l'accuratezza di tale modello?

In [20]:
print_eval(X_val, y_val, model)

   Mean squared error: 30.795
       Relative error: 22.08914%
R-squared coefficient: 0.59062


- L'accuratezza è nettamente peggiore rispetto ai casi precedenti
- In questo caso la regolarizzazione è stata quindi eccessiva

- Cosa succede diminuendo il parametro `alpha`, ovvero il peso della regolarizzazione?

In [21]:
model = Pipeline([
    ("scale", StandardScaler()),
    ("regr", Lasso(alpha=0.2)) # <-- cambiato da 2
])
model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('regr',
                 Lasso(alpha=0.2, copy_X=True, fit_intercept=True,
                       max_iter=1000, normalize=False, positive=False,
                       precompute=False, random_state=None, selection='cyclic',
                       tol=0.0001, warm_start=False))],
         verbose=False)

In [22]:
pd.Series(model.named_steps["regr"].coef_, X.columns)

CRIM      -0.394705
ZN         0.192948
INDUS     -0.000000
CHAS       0.827011
NOX       -0.812358
RM         2.944153
AGE       -0.000000
DIS       -1.514910
RAD        0.000000
TAX       -0.000000
PTRATIO   -1.698942
B          0.842742
LSTAT     -4.009724
dtype: float64

- I coefficienti non nulli sono aumentati da 5 a 11

In [23]:
print_eval(X_val, y_val, model)

   Mean squared error: 22.527
       Relative error: 16.62391%
R-squared coefficient: 0.70053


- L'accuratezza è di poco inferiore a quella ottenuta con gli altri modelli
- Questo modello richiede però solo 9 variabili invece di 13

## Elastic Net

- La regressione _elastic net_ combina insieme le regolarizzazioni L2 e L1 usate in ridge e lasso
- Si applica in scikit-learn tramite la classe `ElasticNet`, per cui l'errore è calcolato come:
$$ E = \underbrace{\frac{1}{2m} ||X\theta - y||_2 ^ 2}_{\text{errore sui dati}} + \underbrace{\alpha \rho ||\theta||_1}_{\text{L1}} + \underbrace{\frac{\alpha(1-\rho)}{2} ||\theta||_2 ^ 2}_{\text{L2}} $$
- I parametri impostabili sono
  - `alpha` ($\alpha$) che determina il peso generale della regolarizzazione
  - `l1_ratio` ($\rho$, compreso tra 0 e 1) che determina il peso di L1 relativo al totale (con $\rho=1$ si ha la regressione lasso, con $\rho=0$ la ridge)

In [24]:
from sklearn.linear_model import ElasticNet
model = Pipeline([
    ("scale",  StandardScaler()),
    ("regr", ElasticNet(alpha=0.2, l1_ratio=0.1))
])
model.fit(X_train, y_train)
print_eval(X_val, y_val, model)

   Mean squared error: 22.092
       Relative error: 16.18298%
R-squared coefficient: 0.70631


## Esercizio 2: Elastic net con pesi separati

- **(2a)** Definire una funzione `elastic_net_with_alphas` che restituisca un modello elastic net (non addestrato) con pesi dati separatamente per la regolarizzazione L2 e L1
  - si ricordi che il parametro `alpha` è la somma dei due pesi
- **(2b)** Servendosi di tale funzione, addestrare e validare un modello elastic net con $\alpha_{L2}=1, \alpha_{L1}=0.1$ e standardizzazione delle feature

In [27]:
def elastic_net_with_alphas(alpha_l2, alpha_l1):
    alpha = alpha_l1 + alpha_l2
    l1_ratio = alpha_l1 / alpha
    return ElasticNet(alpha=alpha, l1_ratio=l1_ratio)

In [28]:
model = Pipeline([
    ("scale", StandardScaler()),
    ("regr",  elastic_net_with_alphas(1, 0.1))
])
model.fit(X_train, y_train)
print_eval(X_val, y_val, model)

   Mean squared error: 28.122
       Relative error: 18.96992%
R-squared coefficient: 0.62615


## Ripasso: Regressione polinomiale univariata

- Abbiamo visto in precedenza la regressione polinomiale su una sola variabile $X$ (univariata), corrispondente alla regressione lineare sulle variabili $X,X^2,X^3,\ldots$
- Per generare queste variabili utilizziamo il filtro `PolynomialFeatures`

In [29]:
from sklearn.preprocessing import PolynomialFeatures

- Siano date ad esempio due osservazioni di una variabile...

In [30]:
sample = np.array([ [ 2],
                    [-3] ])

- Possiamo ottenere ad es. le potenze fino al 4° grado
  - `include_bias=True` specifica di non includere il termine di grado 0

In [31]:
poly = PolynomialFeatures(degree=4, include_bias=False)
poly.fit_transform(sample)
#         X   X^2   X^3   X^4

array([[  2.,   4.,   8.,  16.],
       [ -3.,   9., -27.,  81.]])

## Regressione polinomiale multivariata

- In presenza di più di una variabile, la regressione polinomiale genera tutti i possibili termini fino al grado impostato, includendo anche **termini basati su più variabili**
- Vediamo un esempio con 2 generiche variabili $A$ e $B$

In [32]:
#                     A   B
sample = np.array([ [ 2, -3],
                    [ 4, -5] ])

- Applicando il filtro `PolynomialFeatures` con grado 2...

In [33]:
poly = PolynomialFeatures(degree=2, include_bias=False)
poly.fit_transform(sample)
#         A     B    A^2   A*B   B^2

array([[  2.,  -3.,   4.,  -6.,   9.],
       [  4.,  -5.,  16., -20.,  25.]])

- Le variabili generate (escludendo il grado nullo) sono 5: $A,B,A^2,AB,B^2$
- Oltre ai quadrati delle singole variabili abbiamo quindi anche i prodotti tra di esse

- Possiamo usare il metodo `get_feature_names` del filtro per avere una lista delle variabili calcolate

In [34]:
poly.get_feature_names()

['x0', 'x1', 'x0^2', 'x0 x1', 'x1^2']

- Di default le variabili sono chiamate `x0`, `x1`, ...
- È però possibile specificare i nomi alle variabili

In [35]:
poly.get_feature_names(["A", "B"])

['A', 'B', 'A^2', 'A B', 'B^2']

- Aumentando il grado massimo, le variabili generate **aumentano rapidamente**
- Ad esempio, aumentando il grado da 2 a 3...

In [36]:
poly = PolynomialFeatures(degree=3, include_bias=False)
poly.fit_transform(sample)

array([[   2.,   -3.,    4.,   -6.,    9.,    8.,  -12.,   18.,  -27.],
       [   4.,   -5.,   16.,  -20.,   25.,   64.,  -80.,  100., -125.]])

- ...generiamo 9 variabili, ovvero:

In [37]:
poly.get_feature_names(["A", "B"])

['A', 'B', 'A^2', 'A B', 'B^2', 'A^3', 'A^2 B', 'A B^2', 'B^3']

- Cosa succede con un numero iniziale di variabili più alto?
- Prendiamo ad esempio la matrice `X_train`, con 13 variabili

In [38]:
X_train.shape

(337, 13)

- Generando le feature polinomiali con grado massimo 2...

In [39]:
poly = PolynomialFeatures(degree=2, include_bias=False)
poly.fit_transform(X_train).shape

(337, 104)

- ...otteniamo 104 feature distinte!

- Le feature includono infatti tutte le possibili coppie di variabili, oltre ai quadrati di ciascuna

In [40]:
poly.get_feature_names(X.columns)[-10:]

['TAX^2',
 'TAX PTRATIO',
 'TAX B',
 'TAX LSTAT',
 'PTRATIO^2',
 'PTRATIO B',
 'PTRATIO LSTAT',
 'B^2',
 'B LSTAT',
 'LSTAT^2']

- Aumentando ulteriormente il grado, il numero di variabili cresce enormemente
  - con grado 10 si supera il milione di variabili

In [41]:
poly = PolynomialFeatures(degree=3, include_bias=False)
poly.fit_transform(X_train).shape

(337, 559)

In [42]:
poly = PolynomialFeatures(degree=4, include_bias=False)
poly.fit_transform(X_train).shape

(337, 2379)

In [43]:
poly = PolynomialFeatures(degree=5, include_bias=False)
poly.fit_transform(X_train).shape

(337, 8567)

In [44]:
poly = PolynomialFeatures(degree=6, include_bias=False)
poly.fit_transform(X_train).shape

(337, 27131)

- All'aumentare delle variabili, aumenta il tempo necessario per l'addestramento del modello
- Proviamo ad esempio ad addestrare un modello ElasticNet polinomiale di grado 2, standardizzando le feature generate
- Usiamo il comando "magico" `%time` per riportare in output il tempo di esecuzione

In [45]:
model = Pipeline([
    ("poly",   PolynomialFeatures(degree=2, include_bias=False)),
    ("scale",  StandardScaler()),
    ("regr",   ElasticNet(alpha=0.5, l1_ratio=0.2))
])
%time model.fit(X_train, y_train)
print_eval(X_val, y_val, model)

CPU times: user 12 ms, sys: 4 ms, total: 16 ms
Wall time: 7.99 ms
   Mean squared error: 19.854
       Relative error: 15.44921%
R-squared coefficient: 0.73606


- L'addestramento richiede una frazione di secondo, anche per via del numero limitato di istanze

- Verifichiamo ora cosa accade con grado 5

In [46]:
model = Pipeline([
    ("poly",   PolynomialFeatures(degree=5, include_bias=False)),
    ("scale",  StandardScaler()),
    ("regr",   ElasticNet(alpha=0.5, l1_ratio=0.2))
])
%time model.fit(X_train, y_train)
print_eval(X_val, y_val, model)

CPU times: user 1.55 s, sys: 324 ms, total: 1.88 s
Wall time: 1.55 s
   Mean squared error: 11.969
       Relative error: 11.63031%
R-squared coefficient: 0.84089


/home/rrobby/.venvs/dia2020/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 170.67231189879794, tolerance: 2.9769638753709198
  positive)


- L'accuratezza del modello è migliorata sensibilmente, ma **il tempo per addestrarlo è aumentato di oltre 100 volte**
  - con dataset più grandi, avremmo tempi di addestramento insostenibili

## Esercizio 3: Feature polinomiali e standardizzazione

- **(3a)** Addestrare e validare un modello elastic net `(alpha=0.5, l1_ratio=0.2)` su feature polinomiali di 3° grado, applicando la standardizzazione alle feature originali invece che a quelle generate dal filtro `PolynomialFeatures`
- **(3b)** Addestrare e validare un modello elastic net come sopra, applicando la standardizzazione sia alle feature originali che a quelle polinomiali generate

In [48]:
model = Pipeline([
    ("scale",  StandardScaler()),
    ("poly",   PolynomialFeatures(degree=3, include_bias=False)),
    ("regr",   ElasticNet(alpha=0.5, l1_ratio=0.2))
])
model.fit(X_train, y_train)
print_eval(X_val, y_val, model)

   Mean squared error: 14.398
       Relative error: 12.98614%
R-squared coefficient: 0.80859


In [49]:
model = Pipeline([
    ("scale",  StandardScaler()),
    ("poly",   PolynomialFeatures(degree=3, include_bias=False)),
    ("scale2", StandardScaler()),
    ("regr",   ElasticNet(alpha=0.5, l1_ratio=0.2))
])
model.fit(X_train, y_train)
print_eval(X_val, y_val, model)

   Mean squared error: 15.09
       Relative error: 13.08361%
R-squared coefficient: 0.7994


## Regressione con Funzioni Kernel

- Nella regressione polinomiale si eseguono prodotti tra dati con dimensioni aggiunte e rappresentate esplicitamente
- Le _funzioni kernel_ permettono di calcolare gli stessi prodotti senza calcolare esplicitamente le dimensioni aggiunte
- Questo permette di ottenere modelli non lineari senza l'aggiunta di variabili
- Esistono diverse funzioni kernel utilizzabili con diversi parametri impostabili
- Ad esempio, il kernel polinomiale è definito dalla formula
$$ K(\mathbf{a},\mathbf{b}) = \left(\mathbf{a}\cdot\mathbf{b}+c\right)^d $$
  - $d$ e $c$ sono parametri del kernel, in particolare $d$ è il grado del polinomio

- La classe `KernelRidge` implementa la regressione ridge con l'applicazione di una funzione kernel
- Col parametro `kernel` si indica il tipo di kernel con una stringa, ad es. `"poly"` per un kernel polinomiale
- Ulteriori parametri riguardano il kernel, per quello polinomiale sono `degree` ($d$) e `coef0` ($c$)

In [50]:
from sklearn.kernel_ridge import KernelRidge
model = Pipeline([
    ("scale", StandardScaler()),
    ("regr",  KernelRidge(alpha=20, kernel="poly", degree=5))
])
%time model.fit(X_train, y_train)
print_eval(X_val, y_val, model)

CPU times: user 24 ms, sys: 36 ms, total: 60 ms
Wall time: 22.2 ms
   Mean squared error: 14.765
       Relative error: 13.09973%
R-squared coefficient: 0.80371


- Abbiamo ottenuto un'accuratezza più elevata rispetto ai modelli lineari, ma in tempi molto più brevi rispetto alla regressione polinomiale

- Aumentando arbitrariamente il grado del polinomio, il tempo impiegato per l'addestramento non cambia
  - in questo caso comunque un grado alto non porta ad un modello accurato

In [51]:
from sklearn.kernel_ridge import KernelRidge
model = Pipeline([
    ("scale", StandardScaler()),
    ("regr",  KernelRidge(alpha=20, kernel="poly", degree=20))
])
%time model.fit(X_train, y_train)
print_eval(X_val, y_val, model)

CPU times: user 28 ms, sys: 28 ms, total: 56 ms
Wall time: 17.4 ms
   Mean squared error: 1.9649e+09
       Relative error: 81646.89281%
R-squared coefficient: -2.6122e+07


/home/rrobby/.venvs/dia2020/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=8.55393e-21): result may not be accurate.
  overwrite_a=False)


- Va però ricordato che la complessità cresce quadraticamente col numero di istanze di training
- Ad esempio, addestrando un modello su _tutti_ i dati invece che sul solo training set, quindi sul 50\% di istanze in più...

In [52]:
from sklearn.kernel_ridge import KernelRidge
model = Pipeline([
    ("scale", StandardScaler()),
    ("regr",  KernelRidge(alpha=20, kernel="poly", degree=5))
])
%time model.fit(X, y)
print_eval(X_val, y_val, model)

CPU times: user 40 ms, sys: 52 ms, total: 92 ms
Wall time: 30.2 ms
   Mean squared error: 8.2617
       Relative error: 8.99352%
R-squared coefficient: 0.89017


- ... il tempo necessario è circa il doppio

- Possiamo testare anche funzioni kernel diverse, ad esempio RBF (_radial basis function_)
  - RBF ha valori tanto più elevati quanto più i valori X sono vicini a 0 (ovvero la media, usando dati standardizzati)
- La funzione RBF ha la forma di una gaussiana, di cui si può impostare l'ampiezza col parametro `gamma`

In [53]:
model = Pipeline([
    ("scale", StandardScaler()),
    ("regr",  KernelRidge(alpha=20, kernel="rbf", gamma=0.01))
])
%time model.fit(X_train, y_train)
print_eval(X_val, y_val, model)

CPU times: user 12 ms, sys: 8 ms, total: 20 ms
Wall time: 6.87 ms
   Mean squared error: 43.906
       Relative error: 19.38425%
R-squared coefficient: 0.41632


- In questo caso specifico il kernel RBF non funziona bene tanto quanto il polinomiale

## Esercizio 4: Ripasso cross validation

- **(4a)** Definire un modello di regressione kernel ridge su feature standardizzate con kernel polinomiale di 3° grado e $\alpha=10$
- **(4b)** Eseguire la cross-validation del modello, utilizzando 5 fold dell'intero set di dati (`X` e `y`) generati dall'oggetto `kf` definito sotto
- **(4c)** Calcolare la media e la deviazione standard dei punteggi R² ottenuti dalla validazione di ciascun fold

In [55]:
# import necessari
from sklearn.model_selection import KFold, cross_validate
# generatore di fold casuali da usare
kf = KFold(5, shuffle=True, random_state=42)

In [56]:
model = Pipeline([
    ("scale", StandardScaler()),
    ("regr",  KernelRidge(alpha=10, kernel="poly", degree=3))
])

In [57]:
cv_results = cross_validate(model, X, y, cv=kf)

In [58]:
cv_scores = cv_results["test_score"]
cv_scores.mean(), cv_scores.std()

(0.8382685697714585, 0.050923469784573724)

## Ricerca degli iperparametri con grid search

- Sui modelli utilizzati finora abbiamo impostato manualmente i valori di diversi iperparametri
  - grado della regressione polinomiale, peso della regolarizzazione, ...
- L'accuratezza del modello può dipendere fortemente da questi valori
- Scelto un generico modello da utilizzare (es. regressione polinomiale o kernel ridge), vorremmo **individuare i valori degli iperparametri che ne massimizzino l'accuratezza**
- scikit-learn fornisce un supporto per eseguire automaticamente la cross validation di un modello con diversi valori degli iperparametri tramite la _grid search_

- Consideriamo ad esempio un modello _elastic net_ di cui fissiamo arbitrariamente il parametro `l1_ratio`

In [59]:
model = ElasticNet(l1_ratio=0.2)

- Vorremmo trovare il migliore valore possibile del parametro `alpha` tra un insieme di valori possibili, ovvero:

In [60]:
candidate_alphas = [0.1, 1, 10]

- Creiamo ora la _griglia_ dei parametri, ovvero un dizionario in cui associamo ai nomi dei parametri variabili i valori che possono assumere
- In questo caso abbiamo un unico parametro variabile, `alpha`

In [61]:
grid = {"alpha": candidate_alphas}

- Definiamo ora un modello `GridSearchCV` indicando
  - il modello "base" con i parametri fissati a priori
  - la griglia dei parametri variabili
  - un metodo `cv` di cross validation da usare

In [62]:
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(model, grid, cv=kf)

- Come per i modelli base, usiamo il metodo `fit` per eseguire l'addestramento, passando la matrice X e il vettore y
- Per ogni valore possibile di `alpha`, scikit-learn esegue la cross-validation per calcolare il punteggio R² medio del modello con quel valore di `alpha`

In [63]:
gs.fit(X_train, y_train);

/home/rrobby/.venvs/dia2020/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


- In seguito ai test, il modello impostato viene (di default) riaddestrato su tutti i dati forniti, usando gli iperparametri che han dato il miglior punteggio medio
- Il modello finale è accessibile all'attributo `gs_best_estimator_`

In [64]:
gs.best_estimator_

ElasticNet(alpha=0.1, copy_X=True, fit_intercept=True, l1_ratio=0.2,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

- Dall'attributo `best_params_` possiamo vedere quali sono i valori selezionati dalla griglia degli iperparametri per tale modello

In [65]:
gs.best_params_

{'alpha': 0.1}

- L'oggetto `GridSearchCV` addestrato può essere usato come un normale modello di predizione, le chiamate a `predict` e altri metodi sono girate al `best_estimator_`

In [66]:
# prezzo predetto per la prima riga del validation set
gs.predict(X_val.iloc[[0]])

array([28.35759953])

In [67]:
# equivalente a
gs.best_estimator_.predict(X_val.iloc[[0]])

array([28.35759953])

- Possiamo valutare il modello sul validation set "esterno", non utilizzato nella grid search

In [68]:
print_eval(X_val, y_val, gs)

   Mean squared error: 21.905
       Relative error: 16.80121%
R-squared coefficient: 0.7088


- L'attributo `cv_results_` fornisce risultati dettagliati su tutti gli iperparametri testati

In [69]:
gs.cv_results_

{'mean_fit_time': array([0.00298381, 0.00214591, 0.00212278]),
 'std_fit_time': array([0.00125824, 0.00039233, 0.0005205 ]),
 'mean_score_time': array([0.0047997 , 0.00126004, 0.0009779 ]),
 'std_score_time': array([4.35237515e-03, 4.04896973e-04, 6.35746796e-05]),
 'param_alpha': masked_array(data=[0.1, 1, 10],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'alpha': 0.1}, {'alpha': 1}, {'alpha': 10}],
 'split0_test_score': array([0.72725222, 0.71402229, 0.62476928]),
 'split1_test_score': array([0.49684999, 0.52396629, 0.37364136]),
 'split2_test_score': array([0.63715729, 0.59460273, 0.57229431]),
 'split3_test_score': array([0.72120542, 0.67019429, 0.53969277]),
 'split4_test_score': array([0.80022849, 0.7617415 , 0.62314866]),
 'mean_test_score': array([0.67615597, 0.65270417, 0.54642735]),
 'std_test_score': array([0.1038295 , 0.0848443 , 0.09260055]),
 'rank_test_score': array([1, 2, 3], dtype=int32)}

- Incapsuliamo tali risultati in un DataFrame per visualizzarli meglio ed ordinarli per punteggio decrescente

In [70]:
results = pd.DataFrame(gs.cv_results_)
results.sort_values("mean_test_score", ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002984,0.001258,0.004800,0.004352,0.1,{'alpha': 0.1},0.727252,0.496850,0.637157,0.721205,0.800228,0.676156,0.103829,1
1,0.002146,0.000392,0.001260,0.000405,1,{'alpha': 1},0.714022,0.523966,0.594603,0.670194,0.761741,0.652704,0.084844,2
2,0.002123,0.000520,0.000978,0.000064,10,{'alpha': 10},0.624769,0.373641,0.572294,0.539693,0.623149,0.546427,0.092601,3


I dati riportati per ciascun test includono:
- `{mean|std}_{fit|score}_time`: media/dev. standard dei tempi di addestramento/valutazione sui diversi fold
- `param_X`: valore del parametro X
- `params`: dizionario col valore di tutti i parametri
- `splitN_test_score`: punteggio della valutazione sull'N-esimo fold
- `{mean|std}_test_score`: media/dev. standard dei punteggi sui diversi fold
- `rank_test_score`: ranking del punteggio, 1 è il migliore

- Cosa succede con due iperparametri variabili?
- Oltre a 3 valori possibili per `alpha`, impostiamo 3 valori possibili anche per `l1_ratio`

In [71]:
model = ElasticNet()
grid = {
    "alpha":    [0.1, 1, 10],
    "l1_ratio": [0.1, 0.2, 0.3]
}
gs = GridSearchCV(model, grid, cv=kf)
gs.fit(X_train, y_train);

/home/rrobby/.venvs/dia2020/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [72]:
pd.DataFrame(gs.cv_results_).sort_values("mean_test_score", ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001963,0.000190,0.000972,0.000066,0.1,0.1,"{'alpha': 0.1, 'l1_ratio': 0.1}",0.727791,0.499753,0.636757,0.721171,0.800223,0.676763,0.102900,1
1,0.002242,0.000505,0.001216,0.000466,0.1,0.2,"{'alpha': 0.1, 'l1_ratio': 0.2}",0.727252,0.496850,0.637157,0.721205,0.800228,0.676156,0.103829,2
2,0.001895,0.000071,0.000961,0.000033,0.1,0.3,"{'alpha': 0.1, 'l1_ratio': 0.3}",0.726624,0.493642,0.637503,0.721081,0.800186,0.675417,0.104841,3
3,0.002140,0.000397,0.001241,0.000467,1,0.1,"{'alpha': 1, 'l1_ratio': 0.1}",0.714960,0.526437,0.595148,0.670794,0.763423,0.653954,0.084608,4
4,0.001805,0.000213,0.000927,0.000054,1,0.2,"{'alpha': 1, 'l1_ratio': 0.2}",0.714022,0.523966,0.594603,0.670194,0.761741,0.652704,0.084844,5
5,0.002306,0.000715,0.000929,0.000029,1,0.3,"{'alpha': 1, 'l1_ratio': 0.3}",0.714237,0.521933,0.594699,0.669538,0.759777,0.651835,0.084962,6
6,0.001713,0.000094,0.000927,0.000036,10,0.1,"{'alpha': 10, 'l1_ratio': 0.1}",0.638007,0.397883,0.577252,0.557198,0.655982,0.564984,0.091643,7
7,0.001912,0.000495,0.000892,0.000030,10,0.2,"{'alpha': 10, 'l1_ratio': 0.2}",0.624769,0.373641,0.572294,0.539693,0.623149,0.546427,0.092601,8
8,0.001642,0.000086,0.000878,0.000031,10,0.3,"{'alpha': 10, 'l1_ratio': 0.3}",0.614417,0.357906,0.570172,0.529856,0.607627,0.535700,0.094342,9


- scikit-learn ha generato e testato **tutte le combinazioni possibili** dei valori degli iperparametri, in tutto 3×3 = 9 configurazioni

### Grid search su pipeline

- Possiamo usare `GridSearchCV` anche con una pipeline, testando diversi valori anche per i parametri dei filtri
- Consideriamo ad esempio un modello polinomiale con regolarizzazione L2, su cui sono variabili
  - il grado del polinomio (attributo `degree` del filtro `poly`)
  - il peso della regolarizzazione (attributo `alpha` del modello `regr`)

In [73]:
model = Pipeline([
    ("poly",  PolynomialFeatures(include_bias=False)),
    ("scale", StandardScaler()),
    ("regr",  Ridge())
])

- Per riferirsi ai parametri dei singoli componenti, usiamo la notazione `componente__parametro` _(con DUE underscore in mezzo)_

In [74]:
grid = {
    "poly__degree": [2, 3],      # <- grado polinomio
    "regr__alpha":  [0.1, 1, 10] # <- regolarizzazione
}
gs = GridSearchCV(model, grid, cv=kf)

In [75]:
gs.fit(X_train, y_train);
pd.DataFrame(gs.cv_results_).sort_values("mean_test_score", ascending=False)

/home/rrobby/.venvs/dia2020/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_poly__degree,param_regr__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,0.012698,0.002000,0.002977,0.000900,2,1,"{'poly__degree': 2, 'regr__alpha': 1}",0.835447,0.596491,0.802240,0.810611,0.882463,0.785038,0.098802,1
2,0.015762,0.003819,0.002716,0.000884,2,10,"{'poly__degree': 2, 'regr__alpha': 10}",0.842239,0.594859,0.750900,0.795789,0.856246,0.767713,0.094450,2
0,0.014436,0.005432,0.002357,0.000504,2,0.1,"{'poly__degree': 2, 'regr__alpha': 0.1}",0.824409,0.648065,0.850516,0.816894,0.493138,0.726662,0.136693,3
5,0.027469,0.012958,0.015329,0.009409,3,10,"{'poly__degree': 3, 'regr__alpha': 10}",0.853895,0.454814,0.773790,0.748621,0.333358,0.633023,0.201746,4
4,0.025623,0.002924,0.009970,0.001819,3,1,"{'poly__degree': 3, 'regr__alpha': 1}",0.852441,0.568150,0.794898,0.775824,-3.047350,-0.006925,1.517645,5
3,0.025479,0.001802,0.011557,0.001309,3,0.1,"{'poly__degree': 3, 'regr__alpha': 0.1}",0.692042,0.664729,0.648950,0.784034,-3.348237,-0.107007,1.615276,6


- Nella pipeline possiamo impostare un intero componente come parametro variabile, con la possibilità di rimuoverlo impostandolo a `None`
- Possiamo ad esempio testare un modello con e senza standardizzazione delle feature

In [76]:
model = Pipeline([
    ("scale", None),
    ("regr",  Ridge())
])
grid = {
    # scale = standardizzazione oppure nulla
    "scale": [None, StandardScaler()],
    "regr__alpha": [0.1, 1, 10]
}
gs = GridSearchCV(model, grid, cv=kf)

In [77]:
gs.fit(X_train, y_train);
pd.DataFrame(gs.cv_results_).sort_values("mean_test_score", ascending=False)

/home/rrobby/.venvs/dia2020/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regr__alpha,param_scale,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,0.002675,0.000134,0.001051,0.000047,1,"StandardScaler(copy=True, with_mean=True, with...","{'regr__alpha': 1, 'scale': StandardScaler(cop...",0.735767,0.495302,0.657167,0.720101,0.800925,0.681459,0.104112,1
1,0.006050,0.003837,0.001187,0.000148,0.1,"StandardScaler(copy=True, with_mean=True, with...","{'regr__alpha': 0.1, 'scale': StandardScaler(c...",0.735121,0.495993,0.656523,0.720110,0.801136,0.681383,0.103875,2
5,0.002648,0.000130,0.001044,0.000030,10,"StandardScaler(copy=True, with_mean=True, with...","{'regr__alpha': 10, 'scale': StandardScaler(co...",0.741449,0.487396,0.662300,0.718528,0.798668,0.681269,0.106783,3
0,0.004251,0.002839,0.001726,0.000932,0.1,None,"{'regr__alpha': 0.1, 'scale': None}",0.734675,0.494498,0.655446,0.718577,0.801852,0.680615,0.104470,4
2,0.001795,0.000049,0.000970,0.000084,1,None,"{'regr__alpha': 1, 'scale': None}",0.730686,0.487010,0.649269,0.712555,0.801934,0.675890,0.106636,5
4,0.001774,0.000103,0.000917,0.000024,10,None,"{'regr__alpha': 10, 'scale': None}",0.726584,0.488528,0.641675,0.719819,0.801424,0.675202,0.106541,6


## Esercizio 5: Grid search su regressione kernel ridge

- **(5a)** Creare una funzione `grid_test` che, presi in input un modello e una griglia di parametri
  - esegua una grid search con `X_train`, `y_train` come dati di addestramento con modello e parametri forniti, usando cross-validation a 5 fold
  - stampi (`print(...)`) il dizionario con gli iperparametri selezionati del modello migliore
  - stampi le misure di accuratezza (`print_eval`) sui dati `X_val`, `y_val`
- **(5b)** Testare con la funzione sopra un modello di regressione elastic net polinomiale con
  - grado 2 o 3
  - standardizzazione delle feature generate
  - `alpha` pari a 0.1, 1 o 10
  - `l1_ratio` pari a 0.1, 0.25 o 0.5
- **(5c)** Testare con la funzione sopra un modello kernel ridge con
  - standardizzazione dei dati
  - kernel polinomiale di grado compreso tra 2 e 10
  - `alpha` (regolarizzazione) pari a 0.01, 0.1, 1 o 10

In [80]:
def grid_test(model, grid):
    gs = GridSearchCV(model, grid, cv=kf)
    gs.fit(X_train, y_train)
    print(gs.best_params_)
    print_eval(X_val, y_val, gs)

In [81]:
model = Pipeline([
    ("poly", PolynomialFeatures(include_bias=False)),
    ("scale", StandardScaler()),
    ("regr", ElasticNet())
])
grid = {
    "poly__degree": [2, 3],
    "regr__alpha": [0.1, 1, 10],
    "regr__l1_ratio": [0.1, 0.25, 0.5]
}
grid_test(model, grid)

/home/rrobby/.venvs/dia2020/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 197.98020686962036, tolerance: 2.465989197026022
  positive)
/home/rrobby/.venvs/dia2020/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 52.14783983447796, tolerance: 2.2563256431226772
  positive)
/home/rrobby/.venvs/dia2020/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 63.80528399091236, tolerance: 2.406635866666666
  positive)
/home/rrobby/.venvs/dia2020/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might wa

{'poly__degree': 2, 'regr__alpha': 0.1, 'regr__l1_ratio': 0.1}
   Mean squared error: 15.041
       Relative error: 13.87150%
R-squared coefficient: 0.80004


/home/rrobby/.venvs/dia2020/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/rrobby/.venvs/dia2020/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 278.6543434185819, tolerance: 2.9769638753709198
  positive)


In [82]:
model = Pipeline([
    ("scale", StandardScaler()),
    ("regr", KernelRidge(kernel="poly"))
])
grid = {
    "regr__degree": range(2, 11),
    "regr__alpha": [0.01, 0.1, 1, 10],
}
grid_test(model, grid)

{'regr__alpha': 1, 'regr__degree': 2}
   Mean squared error: 11.303
       Relative error: 12.20756%
R-squared coefficient: 0.84973


/home/rrobby/.venvs/dia2020/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


## Nested cross-validation

- Finora abbiamo validato il risultato finale della grid search su un validation set separato
- La _nested cross-validation_ prevede che siano generati **k fold "esterni"** su tutti i dati disponibili e che **per ciascuno** si esegua il tuning degli iperparametri con una cross validation "interna" usando le parti di training dei fold esterni
- I criteri con cui si eseguono le cross validation esterna ed interna possono differire, es. diverso numero di fold
- Ipotizziamo ad esempio di usare 3 fold esterni e 5 interni

In [83]:
outer_cv = KFold(3, shuffle=True, random_state=42)
inner_cv = KFold(5, shuffle=True, random_state=42)

## Esercizio 6: Nested cross validation

Implementare una funzione `nested_cv` che esegua la nested cross validation
- predisporre una lista vuota in cui salvare i punteggi ottenuti su ogni fold esterno
- usare un ciclo `for` per iterare tutti i fold esterni (T, V) del dataset `X`, `y`
  - usare il metodo `split` di `outer_cv`, che fornisce per ogni fold gli indici delle istanze di training e di validation
- su ciascun fold esterno, eseguire la grid search con modello e parametri dati in ingresso alla funzione sui dati T, applicando `inner_cv` come cross validation
- per ogni modello generato, salvare nella lista di punteggi il R² ottenuto dalla validazione sui dati V
- restituire la lista alla fine del ciclo
- testare la funzione su uno dei modelli usati sopra

In [86]:
def nested_cv(model, grid):
    results = []
    for train_indices, val_indices in outer_cv.split(X, y):
        gs = GridSearchCV(model, grid, cv=inner_cv)
        gs.fit(X.iloc[train_indices], y.iloc[train_indices])
        score = gs.score(X.iloc[val_indices], y.iloc[val_indices])
        results.append(score)
    return results

In [87]:
model = Pipeline([
    ("scale", StandardScaler()),
    ("regr", KernelRidge(kernel="poly"))
])
grid = {
    "regr__degree": range(2, 11),
    "regr__alpha": [0.01, 0.1, 1, 10],
}
nested_cv(model, grid)

/home/rrobby/.venvs/dia2020/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/rrobby/.venvs/dia2020/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/rrobby/.venvs/dia2020/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[0.857809313401843, 0.8443841766460665, 0.8875611674403587]